# Overview

REFERENCE: https://python.langchain.com/docs/use_cases/graph/graph_cypher_qa


## Setup

In [1]:
#!pip install openai

In [2]:
#!pip install python-dotenv

In [3]:
#!pip install  langchain langchain-experimental

In [4]:
#!pip install psycopg2 python-environ

In [5]:
import os
#import openai
from dotenv import load_dotenv, find_dotenv
from langchain.llms import OpenAI
from langchain.chains import GraphCypherQAChain
from langchain.chat_models import ChatOpenAI
from langchain.graphs import Neo4jGraph
from langchain.prompts.prompt import PromptTemplate

_ = load_dotenv(find_dotenv()) 
#openai.api_key  = os.getenv('OPENAI_API_KEY')
API_KEY = os.getenv('OPENAI_API_KEY')
#DBPASS=os.getenv('DBPASS')
#DATABASE=os.getenv('DATABASE')
NEO4J_PASSWORD=os.getenv('NEO4J_PASSWORD')
NEO4J_URL=os.getenv('NEO4J_URL')
NEO4J_DB=os.getenv('NEO4J_DB')


graph = Neo4jGraph(
    url= NEO4J_URL, username="neo4j", password=NEO4J_PASSWORD
)

In [6]:
#print(graph.schema)

In [7]:
#graph.query("match (n:Processor{name:'grafana'})-[r:QUERIES|FORWARDS*]-(m) return n ,r,m")


In [8]:
CYPHER_GENERATION_TEMPLATE = """Task:Generate Cypher statement to query a graph database.
Instructions:
Use only the provided relationship types and properties in the schema.
Do not use any other relationship types or properties that are not provided.
Ignore cases of the names or nouns provided in the question.

Schema:
{schema}


Cypher examples:
# How does grafana query data?
MATCH p=(:Processor)-[:QUERIES 1..6]->()
RETURN p AS path
YIELD nodes, relationships RETURN nodes, relationships
# How does data get into grafana?
match(p)-[r*3]-(q:Processor) return p,q,r
# How does Grafana get created?
MATCH p=(:Users)-[*1..3]-()-[:CREATES]->(q:Processor)
RETURN p AS path
# How does grafana get installed?
MATCH p=(:Users)-[*1..3]-()-[:CREATES]->(q:Processor)
RETURN p AS path
#Why do I not see any data in grafana?
match(p)-[*3]-(q:Processor) return p,q

Note: Do not include any explanations or apologies in your responses.
Do not respond to any questions that might ask anything else than for you to construct a Cypher statement.
Do not include any text except the generated Cypher statement.
Examples: Here are a few examples of generated Cypher statements for particular questions:

The question is:
{question}"""

CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=["schema", "question"], template=CYPHER_GENERATION_TEMPLATE
)

In [9]:
chain = GraphCypherQAChain.from_llm(
    ChatOpenAI(temperature=0), graph=graph, verbose=True,cypher_prompt=CYPHER_GENERATION_PROMPT
    
)

### This does not add much value: return_intermediate_steps=True

In [10]:
#chaindetail = GraphCypherQAChain.from_llm(
#    ChatOpenAI(temperature=0), graph=graph, verbose=True, return_intermediate_steps=True
#)
#result = chaindetail("How does data get into grafana?")
#print(f"Intermediate steps: {result['intermediate_steps']}")
#print(f"Final answer: {result['result']}")

In [11]:
chain.run("How does data get into grafana?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH p=(:Processor)-[*3]-(:Processor {name: "grafana"})
RETURN p AS path
Full Context:
[{'path': [{'namespace': 'open-cluster-management-agent', 'name': 'WorkAgent', 'onHub': 'false', 'type': 'Controller', 'entity': 'Processor', 'onManaged': 'true'}, 'WATCHES', {'namespace': 'managed-cluster-namespace', 'name': 'ManifestWork', 'onHub': 'true', 'type': 'Custom Resource', 'entity': 'IntermediateResource', 'onManaged': 'false'}, 'CREATES', {'namespace': 'open-cluster-management', 'name': 'MulticlusterObservabilityOperator', 'onHub': 'true', 'type': 'Controller', 'entity': 'Processor', 'onManaged': 'false'}, 'CREATES', {'namespace': 'open-cluster-management-observability', 'name': 'grafana', 'onHub': 'true', 'type': 'Pod', 'entity': 'Processor', 'onManaged': 'false'}]}, {'path': [{'namespace': 'open-cluster-management-observability', 'name': 'Thanos Receiver', 'onHub': 'true', 'type': 'Pod', 'entity': 'Processor', 'onManaged':

'Data gets into Grafana through a series of processes. First, the MulticlusterObservabilityOperator in the open-cluster-management namespace creates the Grafana pod in the open-cluster-management-observability namespace. Then, the ObservatoriumOperator in the open-cluster-management-observability namespace creates the Observatorium CR, which is a custom resource. Finally, the ObservatoriumOperator watches the Observatorium CR and creates the necessary resources to collect and store data, which is then visualized in Grafana.'

In [12]:
chain.run("What are the names of the things that Users create?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (u:Users)-[:CREATES]->(n)
RETURN n.name AS createdThing
Full Context:
[{'createdThing': 'Placement'}, {'createdThing': 'PlacementBinding'}, {'createdThing': 'PolicyAutomation'}, {'createdThing': 'Policy'}, {'createdThing': 'ThanosObjectStoreSecret'}, {'createdThing': 'ObservabilityNamespace'}, {'createdThing': 'MCO-CR'}]

> Finished chain.


'The names of the things that Users create are Placement, PlacementBinding, PolicyAutomation, Policy, ThanosObjectStoreSecret, ObservabilityNamespace, and MCO-CR.'

In [13]:
chain.run("How does alert-manager get created?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH p=(:Users)-[*1..3]-()-[:CREATES]->(:API {name: "alert-manager"})
RETURN p AS path
Full Context:
[]

> Finished chain.


"I'm sorry, but I don't have the information to answer your question."

In [14]:
chain.run("Why do I not see any data in grafana")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH p=(:Processor)-[*3]->(:API {name: "grafana"})
RETURN p AS path
Full Context:
[]

> Finished chain.


'There could be several reasons why you are not seeing any data in Grafana. It is possible that there is no data being collected or sent to Grafana from the data source. Another possibility is that there may be an issue with the data source configuration in Grafana, such as incorrect credentials or connection settings. Additionally, it is also worth checking if there are any filters or time range settings applied in Grafana that may be hiding the data.'

In [15]:
chain.run("How does grafana get created?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH p=(:Users)-[*1..3]-()-[:CREATES]->(q:Processor)
RETURN p AS path
Full Context:
[{'path': [{'name': 'User', 'entity': 'Users'}, 'CREATES', {'namespace': 'cluster-scoped', 'name': 'MCO-CR', 'onHub': 'true', 'type': 'Custom Resource', 'entity': 'API', 'onManaged': 'false'}, 'WATCHES', {'namespace': 'open-cluster-management', 'name': 'MulticlusterObservabilityOperator', 'onHub': 'true', 'type': 'Controller', 'entity': 'Processor', 'onManaged': 'false'}, 'CREATES', {'namespace': 'open-cluster-management-observability', 'name': 'alert-manager', 'onHub': 'true', 'type': 'Pod', 'entity': 'Processor', 'onManaged': 'false'}]}, {'path': [{'name': 'User', 'entity': 'Users'}, 'CREATES', {'namespace': 'cluster-scoped', 'name': 'MCO-CR', 'onHub': 'true', 'type': 'Custom Resource', 'entity': 'API', 'onManaged': 'false'}, 'WATCHES', {'namespace': 'open-cluster-management', 'name': 'MulticlusterObservabilityOperator', 'onHub': 'true', 

'Grafana gets created through a series of steps. First, a user creates a custom resource called MCO-CR in the cluster-scoped namespace. Then, the MulticlusterObservabilityOperator, which is a controller in the open-cluster-management namespace, watches for this custom resource. Once the MulticlusterObservabilityOperator detects the creation of the MCO-CR, it creates a pod called grafana in the open-cluster-management-observability namespace.'